In [1]:
from selenium import webdriver as wb
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd

data = []

url = "https://www.oliveyoung.co.kr/store/main/getBestList.do?dispCatNo=900000100100001&fltDispCatNo=10000010007&pageIdx=1&rowsPerPage=8&t_page=%EB%9E%AD%ED%82%B9&t_click=%ED%8C%90%EB%A7%A4%EB%9E%AD%ED%82%B9_%EB%A7%A8%EC%A6%88%EC%BC%80%EC%96%B4"
driver = wb.Chrome()
driver.maximize_window()
driver.get(url)
body = driver.find_element(By.CSS_SELECTOR, 'body')
body.send_keys(Keys.SPACE)
time.sleep(1) 
body.send_keys(Keys.SPACE)
def collect_reviews():
    reviews = []
    for i in range(1, 11):
        try:
            review = driver.find_element(By.CSS_SELECTOR, f"#gdasList > li:nth-child({i}) > div.review_cont > div.txt_inner")
            reviews.append(review.text)
        except Exception as e:
            print(f"{i}번째 리뷰를 찾을 수 없습니다🤖: {e}")
        time.sleep(1)
    return reviews

for i in range(13, 19):
    for j in range(1, 5):
        try:
            rank = driver.find_element(By.CSS_SELECTOR, f"#Container > div.best-area > div.TabsConts.on > ul:nth-child({i}) > li:nth-child({j}) > div > a > img")
            rank.click()
            time.sleep(2)  # Wait for the page to load

            brand = driver.find_element(By.CSS_SELECTOR, "a.pd_arrow_link").text
            item_name = driver.find_element(By.CSS_SELECTOR, "p.prd_name").text
            category = driver.find_element(By.CSS_SELECTOR, "a#dtlCatNm").text

            try:
                price = driver.find_element(By.CSS_SELECTOR, "span.price-1").text
            except:
                price = driver.find_element(By.CSS_SELECTOR, "span.price-2").text
            price = price.replace("\n", "")

            sale_price = driver.find_element(By.CSS_SELECTOR, "span.price-2").text.replace("\n", "")
            stars = driver.find_element(By.CSS_SELECTOR, '#repReview > b').text

            # Navigate to review section
            body = driver.find_element(By.CSS_SELECTOR, 'body')
            body.send_keys(Keys.SPACE)
            time.sleep(1)

            review_section = driver.find_element(By.CSS_SELECTOR, '#reviewInfo > a')
            review_section.click()
            time.sleep(1)

            body.send_keys(Keys.SPACE)
            time.sleep(1)

            try:
                type_element = driver.find_element(By.CSS_SELECTOR, '#gdasList > li:nth-child(1) > div.review_cont > div.poll_sample')
                type_feature = type_element.text.split()
            except Exception as e:
                print(f"Type element not found: {e}")
                type_feature = [] 
                
            helpful = driver.find_element(By.CSS_SELECTOR, "#gdasSort > li:nth-child(2) > a")
            helpful.click()
            time.sleep(1)

            all_reviews = []
            all_reviews += collect_reviews()

            for k in range(2, 11):
                try:
                    page = driver.find_element(By.CSS_SELECTOR, f'#gdasContentsArea > div > div.pageing > a:nth-child({k})')
                    page.click()
                    time.sleep(2)
                    all_reviews += collect_reviews()
                except Exception as e:
                    print(f"{k}번째 페이지를 클릭할 수 없습니다🤖: {e}")
                    break

            max_iterations = 10  # Maximum number of iterations
            iteration = 0  # Initialize iteration counter
            
            while iteration < max_iterations:
                try:
                    next_button = driver.find_element(By.CSS_SELECTOR, '#gdasContentsArea > div > div.pageing > a.next')
                    next_button.click()  # Click the 'next' button
                    time.sleep(1)  # Wait for the page to load

                    for k in range(3, 12):
                        try:
                            page_button = driver.find_element(By.CSS_SELECTOR, f'#gdasContentsArea > div > div.pageing > a:nth-child({k})')
                            page_button.click()  
                            time.sleep(1)  
                            all_reviews += collect_reviews()
                        except Exception as e:
                            print(f"{k}번째 페이지 버튼 클릭 실패🤖: {e}")
                            break
                    iteration += 1  # Increment iteration counter
                except Exception as e:
                    print(f"No more pages🍒: {e}")
                    break  # Exit loop if 'next' button is not found

            data.append({
                '브랜드': brand,
                '상품명': item_name,
                '카테고리': category,
                '가격': price,
                '세일 가격': sale_price,
                '별점': stars,
                '상품 특징': type_feature,
                '리뷰': all_reviews
            })
                
            driver.back()
            time.sleep(2)
        except Exception as e:
            print(f"Error processing item {j} in section {i}: {e}")

df = pd.DataFrame(data)

df.to_csv('review_data.csv', index=False)

print(df)

4번째 리뷰를 찾을 수 없습니다🤖: Message: no such element: Unable to locate element: {"method":"css selector","selector":"#gdasList > li:nth-child(4) > div.review_cont > div.txt_inner"}
  (Session info: chrome=126.0.6478.127); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF75360EEA2+31554]
	(No symbol) [0x00007FF753587ED9]
	(No symbol) [0x00007FF75344872A]
	(No symbol) [0x00007FF753498434]
	(No symbol) [0x00007FF75349853C]
	(No symbol) [0x00007FF7534DF6A7]
	(No symbol) [0x00007FF7534BD06F]
	(No symbol) [0x00007FF7534DC977]
	(No symbol) [0x00007FF7534BCDD3]
	(No symbol) [0x00007FF75348A33B]
	(No symbol) [0x00007FF75348AED1]
	GetHandleVerifier [0x00007FF753918B1D+3217341]
	GetHandleVerifier [0x00007FF753965AE3+3532675]
	GetHandleVerifier [0x00007FF75395B0E0+3489152]
	GetHandleVerifier [0x00007FF7536BE776+750614]
	(No symbol) [0x00007FF75359375F]
	(No symbol) [0x00

In [3]:
import pandas as pd

In [5]:
df = pd.read_csv("C:/Users/un991/새싹2기/project_final/review_data.csv", encoding="utf-8")

In [6]:
df['상품명']

0           [콜라겐]아이디얼 포 맨 퍼펙트 올인원 150mL 단품/기획 (2024 AD)
1            바버501 쉐이빙 부스터 스무디 시카 300ml 스페셜기획(+30ml 증정)
2                              다슈 데일리 비타플렉스 남성청결제 300ml
3     [선착순 올인원 미니+ 바디워시 증정] 헤라 옴므 올인원 스킨 하이드로 트리트먼트 ...
4                        라네즈옴므 블루에너지 2종 기획 (미니어처 2종 포함)
5                              KAI 다리털 숱제거 면도기 2입 레그트리머
6                  [리뉴얼] 비레디 생기립밤 3g 01 화사생기 / 02 내추럴생기
7          차앤박(CNP) 옴므랩 안티에이징 솔루션 110ml 증량기획 (+80ml 증정)
8                                     듀이셀 트루빔 파운데이션 12g
9                              존바바토스 XX 아티산 틸 EDT 125ml
10            피지오겔 포맨 기획 3종 (DMT 플루이드/DMT크림/시카밸런스 플루이드)
11                                    베르사체 에로스 EDT 50ml
12    [더 보이즈 영훈 포토카드 증정] 몽블랑 익스플로러 EDP 60ml / 60ml 기...
13                               레이지소사이어티 시카 쉐이빙젤 250ml
14         다슈 데일리 비타플렉스 올인원 바디워시 기획세트 (본품500ml+50ml 증정)
15                                      우르오스 스킨워시 300ml
16    랩시리즈 데일리 레스큐 페이스 로션 50ml 기획세트 (+워터로션30ml+페이스로션...
17                            필립스 그루밍 멀티 트리머(NT3

In [8]:
from selenium import webdriver as wb
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time

In [9]:
data = []

url = "https://www.oliveyoung.co.kr/store/main/getBestList.do?dispCatNo=900000100100001&fltDispCatNo=10000010007&pageIdx=1&rowsPerPage=8&t_page=%EB%9E%AD%ED%82%B9&t_click=%ED%8C%90%EB%A7%A4%EB%9E%AD%ED%82%B9_%EB%A7%A8%EC%A6%88%EC%BC%80%EC%96%B4"
driver = wb.Chrome()
driver.maximize_window()
driver.get(url)
body = driver.find_element(By.CSS_SELECTOR, 'body')
body.send_keys(Keys.SPACE)
time.sleep(1) 
body.send_keys(Keys.SPACE)
def collect_reviews():
    reviews = []
    for i in range(1, 11):
        try:
            review = driver.find_element(By.CSS_SELECTOR, f"#gdasList > li:nth-child({i}) > div.review_cont > div.txt_inner")
            reviews.append(review.text)
        except Exception as e:
            print(f"{i}번째 리뷰를 찾을 수 없습니다🤖: {e}")
        time.sleep(1)
    return reviews

for i in range(9, 11):
    for j in range(1, 5):
        try:
            rank = driver.find_element(By.CSS_SELECTOR, f"#Container > div.best-area > div.TabsConts.on > ul:nth-child({i}) > li:nth-child({j}) > div > a > img")
            rank.click()
            time.sleep(2)  # Wait for the page to load

            brand = driver.find_element(By.CSS_SELECTOR, "a.pd_arrow_link").text
            item_name = driver.find_element(By.CSS_SELECTOR, "p.prd_name").text
            category = driver.find_element(By.CSS_SELECTOR, "a#dtlCatNm").text

            try:
                price = driver.find_element(By.CSS_SELECTOR, "span.price-1").text
            except:
                price = driver.find_element(By.CSS_SELECTOR, "span.price-2").text
            price = price.replace("\n", "")

            sale_price = driver.find_element(By.CSS_SELECTOR, "span.price-2").text.replace("\n", "")
            stars = driver.find_element(By.CSS_SELECTOR, '#repReview > b').text

            # Navigate to review section
            body = driver.find_element(By.CSS_SELECTOR, 'body')
            body.send_keys(Keys.SPACE)
            time.sleep(1)

            review_section = driver.find_element(By.CSS_SELECTOR, '#reviewInfo > a')
            review_section.click()
            time.sleep(1)

            body.send_keys(Keys.SPACE)
            time.sleep(1)

            try:
                type_element = driver.find_element(By.CSS_SELECTOR, '#gdasList > li:nth-child(1) > div.review_cont > div.poll_sample')
                type_feature = type_element.text.split()
            except Exception as e:
                print(f"Type element not found: {e}")
                type_feature = [] 
                
            helpful = driver.find_element(By.CSS_SELECTOR, "#gdasSort > li:nth-child(2) > a")
            helpful.click()
            time.sleep(1)

            all_reviews = []
            all_reviews += collect_reviews()

            for k in range(2, 11):
                try:
                    page = driver.find_element(By.CSS_SELECTOR, f'#gdasContentsArea > div > div.pageing > a:nth-child({k})')
                    page.click()
                    time.sleep(2)
                    all_reviews += collect_reviews()
                except Exception as e:
                    print(f"{k}번째 페이지를 클릭할 수 없습니다🤖: {e}")
                    break

            max_iterations = 10  # Maximum number of iterations
            iteration = 0  # Initialize iteration counter
            
            while iteration < max_iterations:
                try:
                    next_button = driver.find_element(By.CSS_SELECTOR, '#gdasContentsArea > div > div.pageing > a.next')
                    next_button.click()  # Click the 'next' button
                    time.sleep(1)  # Wait for the page to load

                    for k in range(3, 12):
                        try:
                            page_button = driver.find_element(By.CSS_SELECTOR, f'#gdasContentsArea > div > div.pageing > a:nth-child({k})')
                            page_button.click()  
                            time.sleep(1)  
                            all_reviews += collect_reviews()
                        except Exception as e:
                            print(f"{k}번째 페이지 버튼 클릭 실패🤖: {e}")
                            break
                    iteration += 1  # Increment iteration counter
                except Exception as e:
                    print(f"No more pages🍒: {e}")
                    break  # Exit loop if 'next' button is not found

            data.append({
                '브랜드': brand,
                '상품명': item_name,
                '카테고리': category,
                '가격': price,
                '세일 가격': sale_price,
                '별점': stars,
                '상품 특징': type_feature,
                '리뷰': all_reviews
            })
                
            driver.back()
            time.sleep(2)
        except Exception as e:
            print(f"Error processing item {j} in section {i}: {e}")

df = pd.DataFrame(data)

df.to_csv('review_data.csv', index=False)

print(df)

5번째 리뷰를 찾을 수 없습니다🤖: Message: no such element: Unable to locate element: {"method":"css selector","selector":"#gdasList > li:nth-child(5) > div.review_cont > div.txt_inner"}
  (Session info: chrome=126.0.6478.127); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF6B536EEA2+31554]
	(No symbol) [0x00007FF6B52E7ED9]
	(No symbol) [0x00007FF6B51A872A]
	(No symbol) [0x00007FF6B51F8434]
	(No symbol) [0x00007FF6B51F853C]
	(No symbol) [0x00007FF6B523F6A7]
	(No symbol) [0x00007FF6B521D06F]
	(No symbol) [0x00007FF6B523C977]
	(No symbol) [0x00007FF6B521CDD3]
	(No symbol) [0x00007FF6B51EA33B]
	(No symbol) [0x00007FF6B51EAED1]
	GetHandleVerifier [0x00007FF6B5678B1D+3217341]
	GetHandleVerifier [0x00007FF6B56C5AE3+3532675]
	GetHandleVerifier [0x00007FF6B56BB0E0+3489152]
	GetHandleVerifier [0x00007FF6B541E776+750614]
	(No symbol) [0x00007FF6B52F375F]
	(No symbol) [0x00

- 수집한 데이터 합치기

In [2]:
import pandas as pd
df1 = pd.read_csv("C:/Users/un991/새싹2기/project_final/review_data_1-24.csv", encoding="utf-8")
df2 = pd.read_csv("C:/Users/un991/새싹2기/project_final/review_data_25_28.csv", encoding="utf-8")
df3 = pd.read_csv("C:/Users/un991/새싹2기/project_final/review_data_29_32.csv", encoding="utf-8")
df4 = pd.read_csv("C:/Users/un991/새싹2기/project_final/review_data_33-40.csv", encoding="utf-8")
df5 = pd.read_csv("C:/Users/un991/새싹2기/project_final/review_data_41-48.csv", encoding="utf-8")
df6 = pd.read_csv("C:/Users/un991/새싹2기/project_final/review_data_49-72.csv", encoding="utf-8")
df7 = pd.read_csv("C:/Users/un991/새싹2기/project_final/review_data_yeeun.csv", encoding="utf-8")

In [4]:
data = pd.concat([df1,df2,df3,df4,df5,df6,df7])
data

,브랜드,상품명,카테고리,가격,세일 가격,별점,상품 특징,리뷰,Product Name,Brand,Category,Regular Price,Discount Price,Ratings,Main Features,Reviews
0,니베아,니베아 맨 센서티브 쉐이빙 폼 더블 기획 (200ml+200ml),폼/젤/오일/크림,"11,700원","9,900원",4.8,"['제모/절삭력', '아주', '만족해요', '자극도', '자극없이', '순해요',...","['마침사용하던게 다 떨어져서 더블기획이라. 바로 구매했어요', '더블기획이라 꽤나...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,아이디얼포맨,아이디얼 포 맨 프레시 올인원 젤 로션 1+1 한정기획,올인원,"28,000원","22,400원 ~",4.9,"['피부타입', '복합성에', '좋아요', '피부고민', '보습에', '좋아요', ...",['지성피부인 남자친구 선물해줬는데 유분기가 적어서 좋지만 보습력은 떨어지는거 같다...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,오브제,[7월 올영픽 / 덱스 PICK] 오브제 내추럴 커버 파운데이션 기획세트(+클렌징폼...,파운데이션,"36,800원","29,800원 ~",4.8,"['발색력', '아주', '만족해요', '지속력', '지속이', '오래돼요', '발...",['남자분들 조심해서 사용해야 될 것 같어요\n땀 많이 나시는분들은 거의 90프로의...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,라운드랩,[7월 올영픽/관하살 PICK] 라운드랩 포 맨 소나무 진정 토너/로션 2종세트(+...,스킨케어세트,"43,000원","32,000원",4.9,[],['라운드랩 브랜드 믿고 사용하는 브랜드라 이번에 구매해 보았습니다. 생각보다 크기...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,오브제,[7월 올영픽 / 덱스 PICK] 오브제 내추럴 커버 로션 50g 단품/기획(+미니...,BB/CC,"27,900원","25,900원 ~",4.8,"['발색력', '아주', '만족해요', '커버력', '예상보다', '길어요', '발...","[""남자친구 선물로 제가 주문했어요😙\n(남자친구의 피부타입은 붉고 작은 트러블이 ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23,비오템 옴므,비오템 옴므 아쿠아파워 토너 200ml,스킨/토너,"61,000원","51,850원",4.9,"['피부타입', '복합성에', '좋아요', '피부고민', '보습에', '좋아요', ...","['비오템 아쿠아파워 옴므 제품은 사용감이 좋고 수분충전하기 좋은 기초제품인데요, ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24,레이지소사이어티,레이지소사이어티 시카 쉐이빙젤 프로즌 250ml,폼/젤/오일/크림,"15,900원","15,900원",4.9,"['제모/절삭력', '보통이에요', '자극도', '자극없이', '순해요', '지속력...",['피부에 트러블이 생겨 저자극 제품을 찾다 구매해봤어요. 무난하게 잘 사용 할 수...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25,에이딕트,[NEW] 에이딕트 피그말리온 EDP 50ml,향수,"59,000원","49,900원",4.2,"['향', '보통이에요', '지속력', '예상보다', '짧아요', '포장상태', '...",['무화과향이라고 해서 시향 안하고 바로 질렀는데 너무 무거운 느낌.. 지속력도 짧...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26,다슈,다슈 데일리 비타플렉스 올인원 바디워시 기획세트 (본품500ml+50ml 증정),바디워시/로션,"18,000원","11,310원",4.8,"['세정력', '아주', '만족해요', '향', '아주', '만족해요', '보습력'...","['다른거 쓰고 있지만 친구가 좋다고 해서 구매하고 써보기로 했어요. 괜찮아요', ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
data_= data.drop(['Product Name','Brand', 'Category','Regular Price','Discount Price', 'Ratings','Main Features','Reviews'], axis=1)
data_

,브랜드,상품명,카테고리,가격,세일 가격,별점,상품 특징,리뷰
0,니베아,니베아 맨 센서티브 쉐이빙 폼 더블 기획 (200ml+200ml),폼/젤/오일/크림,"11,700원","9,900원",4.8,"['제모/절삭력', '아주', '만족해요', '자극도', '자극없이', '순해요',...","['마침사용하던게 다 떨어져서 더블기획이라. 바로 구매했어요', '더블기획이라 꽤나..."
1,아이디얼포맨,아이디얼 포 맨 프레시 올인원 젤 로션 1+1 한정기획,올인원,"28,000원","22,400원 ~",4.9,"['피부타입', '복합성에', '좋아요', '피부고민', '보습에', '좋아요', ...",['지성피부인 남자친구 선물해줬는데 유분기가 적어서 좋지만 보습력은 떨어지는거 같다...
2,오브제,[7월 올영픽 / 덱스 PICK] 오브제 내추럴 커버 파운데이션 기획세트(+클렌징폼...,파운데이션,"36,800원","29,800원 ~",4.8,"['발색력', '아주', '만족해요', '지속력', '지속이', '오래돼요', '발...",['남자분들 조심해서 사용해야 될 것 같어요\n땀 많이 나시는분들은 거의 90프로의...
3,라운드랩,[7월 올영픽/관하살 PICK] 라운드랩 포 맨 소나무 진정 토너/로션 2종세트(+...,스킨케어세트,"43,000원","32,000원",4.9,[],['라운드랩 브랜드 믿고 사용하는 브랜드라 이번에 구매해 보았습니다. 생각보다 크기...
4,오브제,[7월 올영픽 / 덱스 PICK] 오브제 내추럴 커버 로션 50g 단품/기획(+미니...,BB/CC,"27,900원","25,900원 ~",4.8,"['발색력', '아주', '만족해요', '커버력', '예상보다', '길어요', '발...","[""남자친구 선물로 제가 주문했어요😙\n(남자친구의 피부타입은 붉고 작은 트러블이 ..."
...,...,...,...,...,...,...,...,...
23,비오템 옴므,비오템 옴므 아쿠아파워 토너 200ml,스킨/토너,"61,000원","51,850원",4.9,"['피부타입', '복합성에', '좋아요', '피부고민', '보습에', '좋아요', ...","['비오템 아쿠아파워 옴므 제품은 사용감이 좋고 수분충전하기 좋은 기초제품인데요, ..."
24,레이지소사이어티,레이지소사이어티 시카 쉐이빙젤 프로즌 250ml,폼/젤/오일/크림,"15,900원","15,900원",4.9,"['제모/절삭력', '보통이에요', '자극도', '자극없이', '순해요', '지속력...",['피부에 트러블이 생겨 저자극 제품을 찾다 구매해봤어요. 무난하게 잘 사용 할 수...
25,에이딕트,[NEW] 에이딕트 피그말리온 EDP 50ml,향수,"59,000원","49,900원",4.2,"['향', '보통이에요', '지속력', '예상보다', '짧아요', '포장상태', '...",['무화과향이라고 해서 시향 안하고 바로 질렀는데 너무 무거운 느낌.. 지속력도 짧...
26,다슈,다슈 데일리 비타플렉스 올인원 바디워시 기획세트 (본품500ml+50ml 증정),바디워시/로션,"18,000원","11,310원",4.8,"['세정력', '아주', '만족해요', '향', '아주', '만족해요', '보습력'...","['다른거 쓰고 있지만 친구가 좋다고 해서 구매하고 써보기로 했어요. 괜찮아요', ..."


In [12]:
data_.to_csv('review_data.csv', index=False, encoding='utf-8')

In [13]:
from selenium import webdriver as wb
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd

data = []

url = "https://www.oliveyoung.co.kr/store/main/getBestList.do?dispCatNo=900000100100001&fltDispCatNo=10000010007&pageIdx=1&rowsPerPage=8&t_page=%EB%9E%AD%ED%82%B9&t_click=%ED%8C%90%EB%A7%A4%EB%9E%AD%ED%82%B9_%EB%A7%A8%EC%A6%88%EC%BC%80%EC%96%B4"
driver = wb.Chrome()
driver.maximize_window()
driver.get(url)
body = driver.find_element(By.CSS_SELECTOR, 'body')
body.send_keys(Keys.SPACE)
time.sleep(1) 
body.send_keys(Keys.SPACE)
def collect_reviews():
    reviews = []
    for i in range(1, 11):
        try:
            review = driver.find_element(By.CSS_SELECTOR, f"#gdasList > li:nth-child({i}) > div.review_cont > div.txt_inner")
            reviews.append(review.text)
        except Exception as e:
            print(f"{i}번째 리뷰를 찾을 수 없습니다🤖: {e}")
        time.sleep(1)
    return reviews

for i in range(2, 3):
    for j in range(2, 4):
        try:
            rank = driver.find_element(By.CSS_SELECTOR, f"#Container > div.best-area > div.TabsConts.on > ul:nth-child({i}) > li:nth-child({j}) > div > a > img")
            rank.click()
            time.sleep(2)  # Wait for the page to load

            brand = driver.find_element(By.CSS_SELECTOR, "a.pd_arrow_link").text
            item_name = driver.find_element(By.CSS_SELECTOR, "p.prd_name").text
            category = driver.find_element(By.CSS_SELECTOR, "a#dtlCatNm").text

            try:
                price = driver.find_element(By.CSS_SELECTOR, "span.price-1").text
            except:
                price = driver.find_element(By.CSS_SELECTOR, "span.price-2").text
            price = price.replace("\n", "")

            sale_price = driver.find_element(By.CSS_SELECTOR, "span.price-2").text.replace("\n", "")
            stars = driver.find_element(By.CSS_SELECTOR, '#repReview > b').text

            # Navigate to review section
            body = driver.find_element(By.CSS_SELECTOR, 'body')
            body.send_keys(Keys.SPACE)
            time.sleep(1)

            review_section = driver.find_element(By.CSS_SELECTOR, '#reviewInfo > a')
            review_section.click()
            time.sleep(1)

            body.send_keys(Keys.SPACE)
            time.sleep(1)

            try:
                type_element = driver.find_element(By.CSS_SELECTOR, '#gdasList > li:nth-child(1) > div.review_cont > div.poll_sample')
                type_feature = type_element.text.split()
            except Exception as e:
                print(f"Type element not found: {e}")
                type_feature = [] 
                
            helpful = driver.find_element(By.CSS_SELECTOR, "#gdasSort > li:nth-child(2) > a")
            helpful.click()
            time.sleep(1)

            all_reviews = []
            all_reviews += collect_reviews()

            for k in range(2, 11):
                try:
                    page = driver.find_element(By.CSS_SELECTOR, f'#gdasContentsArea > div > div.pageing > a:nth-child({k})')
                    page.click()
                    time.sleep(2)
                    all_reviews += collect_reviews()
                except Exception as e:
                    print(f"{k}번째 페이지를 클릭할 수 없습니다🤖: {e}")
                    break

            max_iterations = 10  # Maximum number of iterations
            iteration = 0  # Initialize iteration counter
            
            while iteration < max_iterations:
                try:
                    next_button = driver.find_element(By.CSS_SELECTOR, '#gdasContentsArea > div > div.pageing > a.next')
                    next_button.click()  # Click the 'next' button
                    time.sleep(1)  # Wait for the page to load

                    for k in range(3, 12):
                        try:
                            page_button = driver.find_element(By.CSS_SELECTOR, f'#gdasContentsArea > div > div.pageing > a:nth-child({k})')
                            page_button.click()  
                            time.sleep(1)  
                            all_reviews += collect_reviews()
                        except Exception as e:
                            print(f"{k}번째 페이지 버튼 클릭 실패🤖: {e}")
                            break
                    iteration += 1  # Increment iteration counter
                except Exception as e:
                    print(f"No more pages🍒: {e}")
                    break  # Exit loop if 'next' button is not found

            data.append({
                '브랜드': brand,
                '상품명': item_name,
                '카테고리': category,
                '가격': price,
                '세일 가격': sale_price,
                '별점': stars,
                '상품 특징': type_feature,
                '리뷰': all_reviews
            })
                
            driver.back()
            time.sleep(2)
        except Exception as e:
            print(f"Error processing item {j} in section {i}: {e}")

df = pd.DataFrame(data)

df.to_csv('review_갸스비.csv', index=False)

print(df)

6번째 리뷰를 찾을 수 없습니다🤖: Message: no such element: Unable to locate element: {"method":"css selector","selector":"#gdasList > li:nth-child(6) > div.review_cont > div.txt_inner"}
  (Session info: chrome=126.0.6478.127); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF71439EEA2+31554]
	(No symbol) [0x00007FF714317ED9]
	(No symbol) [0x00007FF7141D872A]
	(No symbol) [0x00007FF714228434]
	(No symbol) [0x00007FF71422853C]
	(No symbol) [0x00007FF71426F6A7]
	(No symbol) [0x00007FF71424D06F]
	(No symbol) [0x00007FF71426C977]
	(No symbol) [0x00007FF71424CDD3]
	(No symbol) [0x00007FF71421A33B]
	(No symbol) [0x00007FF71421AED1]
	GetHandleVerifier [0x00007FF7146A8B1D+3217341]
	GetHandleVerifier [0x00007FF7146F5AE3+3532675]
	GetHandleVerifier [0x00007FF7146EB0E0+3489152]
	GetHandleVerifier [0x00007FF71444E776+750614]
	(No symbol) [0x00007FF71432375F]
	(No symbol) [0x00

In [14]:
df

,브랜드,상품명,카테고리,가격,세일 가격,별점,상품 특징,리뷰
0,갸스비,[7월 올영픽] 갸스비 셋&킵 슈퍼하드 스프레이 263ml (레귤러),스프레이/왁스/포마드,"9,000원","4,900원",4.8,"[광택, 유광이에요, 지속력, 보통이에요, 사용감, 보통이에요]",[친구 선물인데 늘 이것만 쓰고 항상 만족한다고 합니다! 갸스비 유명한 이유가 있겠...
1,갸스비,[7월 올영픽] 갸스비 슈퍼하드 스타일링 폼 185g,스프레이/왁스/포마드,"8,800원","4,900원",4.7,"[광택, 무광이에요, 지속력, 보통이에요, 사용감, 보통이에요]",[저희 아빠는 제가 어릴때부터 늘 헤어왁스는 무스타입만 사용하셨어요 ..! 어린 시...


In [15]:
from selenium import webdriver as wb
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd

data = []

url = "https://www.oliveyoung.co.kr/store/main/getBestList.do?dispCatNo=900000100100001&fltDispCatNo=10000010007&pageIdx=1&rowsPerPage=8&t_page=%EB%9E%AD%ED%82%B9&t_click=%ED%8C%90%EB%A7%A4%EB%9E%AD%ED%82%B9_%EB%A7%A8%EC%A6%88%EC%BC%80%EC%96%B4"
driver = wb.Chrome()
driver.maximize_window()
driver.get(url)
body = driver.find_element(By.CSS_SELECTOR, 'body')
body.send_keys(Keys.SPACE)
time.sleep(1) 
body.send_keys(Keys.SPACE)
def collect_reviews():
    reviews = []
    for i in range(1, 11):
        try:
            review = driver.find_element(By.CSS_SELECTOR, f"#gdasList > li:nth-child({i}) > div.review_cont > div.txt_inner")
            reviews.append(review.text)
        except Exception as e:
            print(f"{i}번째 리뷰를 찾을 수 없습니다🤖: {e}")
        time.sleep(1)
    return reviews

for i in range(1, 2):
    for j in range(1, 2):
        try:
            rank = driver.find_element(By.CSS_SELECTOR, f"#Container > div.best-area > div.TabsConts.on > ul:nth-child({i}) > li:nth-child({j}) > div > a > img")
            rank.click()
            time.sleep(2)  # Wait for the page to load

            brand = driver.find_element(By.CSS_SELECTOR, "a.pd_arrow_link").text
            item_name = driver.find_element(By.CSS_SELECTOR, "p.prd_name").text
            category = driver.find_element(By.CSS_SELECTOR, "a#dtlCatNm").text

            try:
                price = driver.find_element(By.CSS_SELECTOR, "span.price-1").text
            except:
                price = driver.find_element(By.CSS_SELECTOR, "span.price-2").text
            price = price.replace("\n", "")

            sale_price = driver.find_element(By.CSS_SELECTOR, "span.price-2").text.replace("\n", "")
            stars = driver.find_element(By.CSS_SELECTOR, '#repReview > b').text

            # Navigate to review section
            body = driver.find_element(By.CSS_SELECTOR, 'body')
            body.send_keys(Keys.SPACE)
            time.sleep(1)

            review_section = driver.find_element(By.CSS_SELECTOR, '#reviewInfo > a')
            review_section.click()
            time.sleep(1)

            body.send_keys(Keys.SPACE)
            time.sleep(1)

            try:
                type_element = driver.find_element(By.CSS_SELECTOR, '#gdasList > li:nth-child(1) > div.review_cont > div.poll_sample')
                type_feature = type_element.text.split()
            except Exception as e:
                print(f"Type element not found: {e}")
                type_feature = [] 
                
            helpful = driver.find_element(By.CSS_SELECTOR, "#gdasSort > li:nth-child(2) > a")
            helpful.click()
            time.sleep(1)

            all_reviews = []
            all_reviews += collect_reviews()

            for k in range(2, 11):
                try:
                    page = driver.find_element(By.CSS_SELECTOR, f'#gdasContentsArea > div > div.pageing > a:nth-child({k})')
                    page.click()
                    time.sleep(2)
                    all_reviews += collect_reviews()
                except Exception as e:
                    print(f"{k}번째 페이지를 클릭할 수 없습니다🤖: {e}")
                    break

            max_iterations = 10  # Maximum number of iterations
            iteration = 0  # Initialize iteration counter
            
            while iteration < max_iterations:
                try:
                    next_button = driver.find_element(By.CSS_SELECTOR, '#gdasContentsArea > div > div.pageing > a.next')
                    next_button.click()  # Click the 'next' button
                    time.sleep(1)  # Wait for the page to load

                    for k in range(3, 12):
                        try:
                            page_button = driver.find_element(By.CSS_SELECTOR, f'#gdasContentsArea > div > div.pageing > a:nth-child({k})')
                            page_button.click()  
                            time.sleep(1)  
                            all_reviews += collect_reviews()
                        except Exception as e:
                            print(f"{k}번째 페이지 버튼 클릭 실패🤖: {e}")
                            break
                    iteration += 1  # Increment iteration counter
                except Exception as e:
                    print(f"No more pages🍒: {e}")
                    break  # Exit loop if 'next' button is not found

            data.append({
                '브랜드': brand,
                '상품명': item_name,
                '카테고리': category,
                '가격': price,
                '세일 가격': sale_price,
                '별점': stars,
                '상품 특징': type_feature,
                '리뷰': all_reviews
            })
                
            driver.back()
            time.sleep(2)
        except Exception as e:
            print(f"Error processing item {j} in section {i}: {e}")

df = pd.DataFrame(data)

df.to_csv('review_갸스비.csv', index=False)

print(df)

1번째 리뷰를 찾을 수 없습니다🤖: Message: no such element: Unable to locate element: {"method":"css selector","selector":"#gdasList > li:nth-child(1) > div.review_cont > div.txt_inner"}
  (Session info: chrome=126.0.6478.127); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF71439EEA2+31554]
	(No symbol) [0x00007FF714317ED9]
	(No symbol) [0x00007FF7141D872A]
	(No symbol) [0x00007FF714228434]
	(No symbol) [0x00007FF71422853C]
	(No symbol) [0x00007FF71426F6A7]
	(No symbol) [0x00007FF71424D06F]
	(No symbol) [0x00007FF71426C977]
	(No symbol) [0x00007FF71424CDD3]
	(No symbol) [0x00007FF71421A33B]
	(No symbol) [0x00007FF71421AED1]
	GetHandleVerifier [0x00007FF7146A8B1D+3217341]
	GetHandleVerifier [0x00007FF7146F5AE3+3532675]
	GetHandleVerifier [0x00007FF7146EB0E0+3489152]
	GetHandleVerifier [0x00007FF71444E776+750614]
	(No symbol) [0x00007FF71432375F]
	(No symbol) [0x00

In [17]:
df_ = pd.read_csv("C:/Users/un991/새싹2기/project_final/review_갸스비1.csv", encoding="utf-8")

In [18]:
gatsby = pd.concat([df,df_])

In [19]:
gatsby

,브랜드,상품명,카테고리,가격,세일 가격,별점,상품 특징,리뷰
0,갸스비,[7월 올영픽/ 1+1 한정기획] 갸스비 매트&하드 스프레이 200ml,스프레이/왁스/포마드,"9,000원","8,550원",4.7,"[광택, 무광이에요, 지속력, 지속이, 오래돼요, 사용감, 아주, 만족해요]",[사용해봤던 스프레이들 중에서는\n이 매트&하드 제품이 가장 마음에 들었는데\n향도...
0,갸스비,[7월 올영픽] 갸스비 셋&킵 슈퍼하드 스프레이 263ml (레귤러),스프레이/왁스/포마드,"9,000원","4,900원",4.8,"['광택', '유광이에요', '지속력', '보통이에요', '사용감', '보통이에요']",['친구 선물인데 늘 이것만 쓰고 항상 만족한다고 합니다! 갸스비 유명한 이유가 있...
1,갸스비,[7월 올영픽] 갸스비 슈퍼하드 스타일링 폼 185g,스프레이/왁스/포마드,"8,800원","4,900원",4.7,"['광택', '무광이에요', '지속력', '보통이에요', '사용감', '보통이에요']","[""저희 아빠는 제가 어릴때부터 늘 헤어왁스는 무스타입만 사용하셨어요 ..! 어린 ..."


In [20]:
gatsby.to_csv('all_gatsby.csv', index=False, encoding="utf-8")